In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [ ]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the dataset
dataset = load_dataset("Anthropic/hh-rlhf")

# Preparing the dataset might require specific adjustments, such as formatting
# the inputs and labels in a way that the model can understand.
train_dataset = dataset['train'].map(
    lambda examples: {'labels': examples['labels']}, batched=True)

# Load the pretrained DeBERTa model
model = DebertaV2ForSequenceClassification.from_pretrained("microsoft/deberta-v3-large")
model.to(device)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory for model and logs
    evaluation_strategy="epoch",     # evaluate each epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset if you have a validation set
)

In [ ]:
# Start training
trainer.train()

# Save the model
model.save_pretrained("./trained_model")